In [1]:
import tensorflow as tf
physcial_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physcial_devices[0],True)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("sign_mnist_train.csv.zip")

In [4]:
X = df.drop(columns='label')
y = df['label']

In [5]:
X.shape

(27455, 784)

In [6]:
y.shape

(27455,)

In [7]:
y = pd.get_dummies(y)

In [8]:
y.shape

(27455, 24)

In [9]:
X_numpy = X.to_numpy()
y_numpy = y.to_numpy()


In [10]:
X_numpy = np.reshape(X_numpy,(X_numpy.shape[0],28,28))

In [11]:
X_numpy.shape

(27455, 28, 28)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# build a sequential model
model = Sequential()
model.add(Rescaling(1/255, input_shape=X_numpy.shape[1:]))

# 1st conv block
model.add(Conv1D(25, 5, activation='relu', strides=1, padding='same'))
model.add(MaxPool1D(pool_size=2, padding='same'))
# 2nd conv block
model.add(Conv1D(50, 5, activation='relu', strides=2, padding='same'))
model.add(MaxPool1D(pool_size=2, padding='same'))
model.add(BatchNormalization())
# 3rd conv block
model.add(Conv1D(70, 3, activation='relu', strides=2, padding='same'))
model.add(MaxPool1D(pool_size=2, padding='valid'))
model.add(BatchNormalization())
# ANN block
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.25))
# output layer
model.add(Dense(units=24, activation='softmax'))


In [13]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [14]:
# display(model.input_shape)
# display(model.output_shape)
# model.summary()

In [15]:
# fit on data for 30 epochs
from tensorflow.keras.callbacks import EarlyStopping
model.fit(X_numpy, y_numpy, epochs=30, validation_split=0.1, batch_size=4,callbacks=[EarlyStopping(patience=2)])

Epoch 1/30
6178/6178 [==============================] - 51s 8ms/step - loss: 2.2445 - accuracy: 0.3103 - val_loss: 0.8394 - val_accuracy: 0.7130
Epoch 2/30
6178/6178 [==============================] - 48s 8ms/step - loss: 1.1269 - accuracy: 0.6285 - val_loss: 0.8948 - val_accuracy: 0.7105
Epoch 3/30
6178/6178 [==============================] - 48s 8ms/step - loss: 0.8697 - accuracy: 0.7030 - val_loss: 0.3565 - val_accuracy: 0.8871
Epoch 4/30
6178/6178 [==============================] - 48s 8ms/step - loss: 0.7080 - accuracy: 0.7578 - val_loss: 0.2231 - val_accuracy: 0.9202
Epoch 5/30
6178/6178 [==============================] - 48s 8ms/step - loss: 0.6263 - accuracy: 0.7884 - val_loss: 0.1629 - val_accuracy: 0.9461
Epoch 6/30
6178/6178 [==============================] - 48s 8ms/step - loss: 0.5622 - accuracy: 0.8133 - val_loss: 0.2025 - val_accuracy: 0.9355
Epoch 7/30
6178/6178 [==============================] - 48s 8ms/step - loss: 0.4807 - accuracy: 0.8411 - val_loss: 0.6329 - val_ac

In [16]:
test_df = pd.read_csv("sign_mnist_test.csv.zip")

In [17]:
X_test = test_df.drop(columns='label')
y_test = test_df['label']

In [18]:
y_test = pd.get_dummies(y_test)

In [19]:
assert y.shape[1] == y_test.shape[1]

In [20]:
X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()


In [21]:
X_test_np = np.reshape(X_test_np,(X_test_np.shape[0],28,28))

In [22]:
X_test_np.shape

(7172, 28, 28)

In [23]:
y_test_np.shape

(7172, 24)

In [24]:
loss,acc = model.evaluate(X_test_np,y_test_np)
print('Test set accuracy:',acc)

225/225 [==============================] - 1s 4ms/step - loss: 1.4932 - accuracy: 0.7125
Test set accuracy: 0.7124930024147034


In [25]:
# print(metrics.classification_report(y_test, y_pred))
y_pred = model.predict(X_test_np)

In [27]:
preds = pd.DataFrame(y_pred,columns=y_test.columns)

In [28]:
affected = (preds != 0).any(axis=1)
nz = preds[affected]
preds[affected] = (nz.T == nz.max(axis=1)).T.astype(int)

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.85      0.80      0.82       331
           1       0.83      1.00      0.91       432
           2       1.00      0.85      0.92       310
           3       0.80      0.96      0.88       245
           4       0.75      1.00      0.85       498
           5       0.76      0.98      0.85       247
           6       0.98      0.43      0.60       348
           7       0.98      0.77      0.86       436
           8       0.62      0.81      0.70       288
           9       0.77      0.53      0.62       331
          10       0.95      0.80      0.87       209
          11       0.77      0.67      0.72       394
          12       0.81      0.16      0.26       291
          13       0.67      0.78      0.72       246
          14       0.91      0.91      0.91       347
          15       0.78      0.91      0.84       164
          16       0.21      0.05      0.08       144
          17       0.74    

We might need to tweak the model to our problem.